# Import Data to SQL with Pandas

In part two of the project, since there are many very large size datasets. I choose to import them into SQL so that it will easier to work with.

Import all libarys

In [1]:
import sys
import pandas as pd
from os import listdir
from os.path import isfile, join
import psycopg2
mypath = "E:\\ForexDataXlsx"


Before Executing, we shall set our confidential data to our env variable.

In [ ]:
import os
#Lets say you have set your env variable in this way,
os.environ["SQL_PASSWORD"] = "passwordXXXXX"
os.environ["SQL_USER"] = "username"

In [ ]:
#Then, 
password = os.environ.get('SQL_PASSWORD')
user = os.environ.get('SQL_USER')

Here I define two function. One will takes a table name as input and create tables and the other will import data to it.
I am using postgreSQL.

In [2]:
def create_table(name, user, password):
    conn = psycopg2.connect(database="Forex", user = user , password=password, host="127.0.0.1", port="5432")
    print("Opened database successfully")

    cur = conn.cursor()
    create_query = "CREATE TABLE " +str(name)+"""( ID INTEGER PRIMARY KEY NOT NULL,
           Datetime TIMESTAMP     NOT NULL,
           Open           FLOAT(8)    NOT NULL,
           High            FLOAT(8)     NOT NULL,
           Low        FLOAT(8)  NOT NULL,
           Close         FLOAT(8)  NOT NULL,
           Vol         FLOAT(8));
    """
    cur.execute(create_query)
    print("Table created successfully")

    conn.commit()
    conn.close()
    
def insert_forex_data(df, name, user, password):
    conn = psycopg2.connect(database="Forex", user = user, password = password, host="127.0.0.1", port="5432")
    cur = conn.cursor()
    insert_query = "INSERT INTO "+ str(name) +" (ID, Datetime, Open, High, Low, Close, Vol) VALUES (%s, %s, %s, %s, %s, %s, %s)"

    for i in df.index:
        r = df.iloc[i,:]

        ids = str(r[0])
        times = str(r[1])
        opens = str(r[2])
        highs = str(r[3])
        lows =  str(r[4]) 
        closes = str(r[5])
        vols =  str(r[6])
        
        values = (ids, times, opens, highs, lows, closes, vols)

        cur.execute(insert_query, values)


    conn.commit()
    conn.close()

Finally, follow the logic would load all the data to SQL. Make sure you have more than 10 GB local disk space.

In [ ]:
onlyfiles = [f for f in listdir(mypath) if ".xlsx" in f][158:]
latest_pair_name = ""
i = 160
length = len(onlyfiles)
for file in onlyfiles:
    
    pair_name = file.split('_')[2]
    print("Current Progress----  ", i ," / ", length, "  ------", "latest_pair_name: ", latest_pair_name, "| current_pair_name: ", pair_name)
    
    if latest_pair_name == "":
        print("Block 1 Execute")
        data = pd.read_excel("E:\\ForexDataXlsx\\" + file ,index_col=None, header=None)
        data[0] = pd.to_datetime(data[0])
        print("File ", file, " has successfully read.")
        
        latest_pair_name = pair_name
        i += 1
        continue
        
    
    if latest_pair_name == pair_name:
        print("Block 2 Execute")
        next_data = pd.read_excel("E:\\ForexDataXlsx\\" + file ,index_col=None, header=None)
        next_data[0] = pd.to_datetime(next_data[0])
        
        data = pd.concat([data, next_data], axis=0)
        print("File ", file, " has successfully read and concat.")
        
        if file == onlyfiles[-1]:
            print("Block 2.2 Execute")
            data = data.reset_index().reset_index().drop('index',axis=1)
            data.columns = [0,1,2,3,4,5,6]
            create_table(pair_name, user, password)
            insert_forex_data(data, pair_name, user, password)
            print(pair_name, "has import to SQL Database.")
    
    else:
        print("Block 3 Execute")
        data = data.reset_index().reset_index().drop('index',axis=1)
        data.columns = [0,1,2,3,4,5,6]
        create_table(latest_pair_name, user, password)
        insert_forex_data(data, latest_pair_name, user, password)
        print(latest_pair_name, "has import to SQL Database.")
        
        
        data = pd.read_excel("E:\\ForexDataXlsx\\" + file ,index_col=None, header=None)
        data[0] = pd.to_datetime(data[0])   
        print("File ", file, " has successfully read.")
        
        
    latest_pair_name = pair_name
    i += 1
    
    
    


Current Progress----   160  /  1212   ------ latest_pair_name:   | current_pair_name:  CADJPY
Block 1 Execute
File  DAT_XLSX_CADJPY_M1_2007.xlsx  has successfully read.
Current Progress----   161  /  1212   ------ latest_pair_name:  CADJPY | current_pair_name:  CADJPY
Block 2 Execute
File  DAT_XLSX_CADJPY_M1_2008.xlsx  has successfully read and concat.
Current Progress----   162  /  1212   ------ latest_pair_name:  CADJPY | current_pair_name:  CADJPY
Block 2 Execute
File  DAT_XLSX_CADJPY_M1_2009.xlsx  has successfully read and concat.
Current Progress----   163  /  1212   ------ latest_pair_name:  CADJPY | current_pair_name:  CADJPY
Block 2 Execute
File  DAT_XLSX_CADJPY_M1_2010.xlsx  has successfully read and concat.
Current Progress----   164  /  1212   ------ latest_pair_name:  CADJPY | current_pair_name:  CADJPY
Block 2 Execute
File  DAT_XLSX_CADJPY_M1_2011.xlsx  has successfully read and concat.
Current Progress----   165  /  1212   ------ latest_pair_name:  CADJPY | current_pair_n

File  DAT_XLSX_CHFJPY_M1_201904.xlsx  has successfully read and concat.
Current Progress----   204  /  1212   ------ latest_pair_name:  CHFJPY | current_pair_name:  CHFJPY
Block 2 Execute
File  DAT_XLSX_CHFJPY_M1_201905.xlsx  has successfully read and concat.
Current Progress----   205  /  1212   ------ latest_pair_name:  CHFJPY | current_pair_name:  CHFJPY
Block 2 Execute
File  DAT_XLSX_CHFJPY_M1_201906.xlsx  has successfully read and concat.
Current Progress----   206  /  1212   ------ latest_pair_name:  CHFJPY | current_pair_name:  CHFJPY
Block 2 Execute
File  DAT_XLSX_CHFJPY_M1_201907.xlsx  has successfully read and concat.
Current Progress----   207  /  1212   ------ latest_pair_name:  CHFJPY | current_pair_name:  CHFJPY
Block 2 Execute
File  DAT_XLSX_CHFJPY_M1_201908.xlsx  has successfully read and concat.
Current Progress----   208  /  1212   ------ latest_pair_name:  CHFJPY | current_pair_name:  CHFJPY
Block 2 Execute
File  DAT_XLSX_CHFJPY_M1_201909.xlsx  has successfully read 

File  DAT_XLSX_EURAUD_M1_201909.xlsx  has successfully read and concat.
Current Progress----   248  /  1212   ------ latest_pair_name:  EURAUD | current_pair_name:  EURAUD
Block 2 Execute
File  DAT_XLSX_EURAUD_M1_201910.xlsx  has successfully read and concat.
Current Progress----   249  /  1212   ------ latest_pair_name:  EURAUD | current_pair_name:  EURAUD
Block 2 Execute
File  DAT_XLSX_EURAUD_M1_201911.xlsx  has successfully read and concat.
Current Progress----   250  /  1212   ------ latest_pair_name:  EURAUD | current_pair_name:  EURCAD
Block 3 Execute
Opened database successfully
Table created successfully
EURAUD has import to SQL Database.
File  DAT_XLSX_EURCAD_M1_2008.xlsx  has successfully read.
Current Progress----   251  /  1212   ------ latest_pair_name:  EURCAD | current_pair_name:  EURCAD
Block 2 Execute
File  DAT_XLSX_EURCAD_M1_2009.xlsx  has successfully read and concat.
Current Progress----   252  /  1212   ------ latest_pair_name:  EURCAD | current_pair_name:  EURCAD


File  DAT_XLSX_EURCHF_M1_201905.xlsx  has successfully read and concat.
Current Progress----   292  /  1212   ------ latest_pair_name:  EURCHF | current_pair_name:  EURCHF
Block 2 Execute
File  DAT_XLSX_EURCHF_M1_201906.xlsx  has successfully read and concat.
Current Progress----   293  /  1212   ------ latest_pair_name:  EURCHF | current_pair_name:  EURCHF
Block 2 Execute
File  DAT_XLSX_EURCHF_M1_201907.xlsx  has successfully read and concat.
Current Progress----   294  /  1212   ------ latest_pair_name:  EURCHF | current_pair_name:  EURCHF
Block 2 Execute
File  DAT_XLSX_EURCHF_M1_201908.xlsx  has successfully read and concat.
Current Progress----   295  /  1212   ------ latest_pair_name:  EURCHF | current_pair_name:  EURCHF
Block 2 Execute
File  DAT_XLSX_EURCHF_M1_201909.xlsx  has successfully read and concat.
Current Progress----   296  /  1212   ------ latest_pair_name:  EURCHF | current_pair_name:  EURCHF
Block 2 Execute
File  DAT_XLSX_EURCHF_M1_201910.xlsx  has successfully read 

File  DAT_XLSX_EURGBP_M1_2014.xlsx  has successfully read and concat.
Current Progress----   335  /  1212   ------ latest_pair_name:  EURGBP | current_pair_name:  EURGBP
Block 2 Execute
File  DAT_XLSX_EURGBP_M1_2015.xlsx  has successfully read and concat.
Current Progress----   336  /  1212   ------ latest_pair_name:  EURGBP | current_pair_name:  EURGBP
Block 2 Execute
File  DAT_XLSX_EURGBP_M1_2016.xlsx  has successfully read and concat.
Current Progress----   337  /  1212   ------ latest_pair_name:  EURGBP | current_pair_name:  EURGBP
Block 2 Execute
File  DAT_XLSX_EURGBP_M1_2017.xlsx  has successfully read and concat.
Current Progress----   338  /  1212   ------ latest_pair_name:  EURGBP | current_pair_name:  EURGBP
Block 2 Execute
File  DAT_XLSX_EURGBP_M1_2018.xlsx  has successfully read and concat.
Current Progress----   339  /  1212   ------ latest_pair_name:  EURGBP | current_pair_name:  EURGBP
Block 2 Execute
File  DAT_XLSX_EURGBP_M1_201901.xlsx  has successfully read and concat

...

File  DAT_XLSX_EURNZD_M1_201905.xlsx  has successfully read and concat.
Current Progress----   421  /  1212   ------ latest_pair_name:  EURNZD | current_pair_name:  EURNZD
Block 2 Execute
File  DAT_XLSX_EURNZD_M1_201906.xlsx  has successfully read and concat.
Current Progress----   422  /  1212   ------ latest_pair_name:  EURNZD | current_pair_name:  EURNZD
Block 2 Execute
File  DAT_XLSX_EURNZD_M1_201907.xlsx  has successfully read and concat.
Current Progress----   423  /  1212   ------ latest_pair_name:  EURNZD | current_pair_name:  EURNZD
Block 2 Execute
File  DAT_XLSX_EURNZD_M1_201908.xlsx  has successfully read and concat.
Current Progress----   424  /  1212   ------ latest_pair_name:  EURNZD | current_pair_name:  EURNZD
Block 2 Execute
File  DAT_XLSX_EURNZD_M1_201909.xlsx  has successfully read and concat.
Current Progress----   425  /  1212   ------ latest_pair_name:  EURNZD | current_pair_name:  EURNZD
Block 2 Execute
File  DAT_XLSX_EURNZD_M1_201910.xlsx  has successfully read 

File  DAT_XLSX_EURSEK_M1_201906.xlsx  has successfully read and concat.
Current Progress----   464  /  1212   ------ latest_pair_name:  EURSEK | current_pair_name:  EURSEK
Block 2 Execute
File  DAT_XLSX_EURSEK_M1_201907.xlsx  has successfully read and concat.
Current Progress----   465  /  1212   ------ latest_pair_name:  EURSEK | current_pair_name:  EURSEK
Block 2 Execute
File  DAT_XLSX_EURSEK_M1_201908.xlsx  has successfully read and concat.
Current Progress----   466  /  1212   ------ latest_pair_name:  EURSEK | current_pair_name:  EURSEK
Block 2 Execute
File  DAT_XLSX_EURSEK_M1_201909.xlsx  has successfully read and concat.
Current Progress----   467  /  1212   ------ latest_pair_name:  EURSEK | current_pair_name:  EURSEK
Block 2 Execute
File  DAT_XLSX_EURSEK_M1_201910.xlsx  has successfully read and concat.
Current Progress----   468  /  1212   ------ latest_pair_name:  EURSEK | current_pair_name:  EURSEK
Block 2 Execute
File  DAT_XLSX_EURSEK_M1_201911.xlsx  has successfully read 

File  DAT_XLSX_EURUSD_M1_201906.xlsx  has successfully read and concat.
Current Progress----   507  /  1212   ------ latest_pair_name:  EURUSD | current_pair_name:  EURUSD
Block 2 Execute
File  DAT_XLSX_EURUSD_M1_201907.xlsx  has successfully read and concat.
Current Progress----   508  /  1212   ------ latest_pair_name:  EURUSD | current_pair_name:  EURUSD
Block 2 Execute
File  DAT_XLSX_EURUSD_M1_201908.xlsx  has successfully read and concat.
Current Progress----   509  /  1212   ------ latest_pair_name:  EURUSD | current_pair_name:  EURUSD
Block 2 Execute
File  DAT_XLSX_EURUSD_M1_201909.xlsx  has successfully read and concat.
Current Progress----   510  /  1212   ------ latest_pair_name:  EURUSD | current_pair_name:  EURUSD
Block 2 Execute
File  DAT_XLSX_EURUSD_M1_201910.xlsx  has successfully read and concat.
Current Progress----   511  /  1212   ------ latest_pair_name:  EURUSD | current_pair_name:  EURUSD
Block 2 Execute
File  DAT_XLSX_EURUSD_M1_201911.xlsx  has successfully read 

File  DAT_XLSX_GBPAUD_M1_201908.xlsx  has successfully read and concat.
Current Progress----   550  /  1212   ------ latest_pair_name:  GBPAUD | current_pair_name:  GBPAUD
Block 2 Execute
File  DAT_XLSX_GBPAUD_M1_201909.xlsx  has successfully read and concat.
Current Progress----   551  /  1212   ------ latest_pair_name:  GBPAUD | current_pair_name:  GBPAUD
Block 2 Execute
File  DAT_XLSX_GBPAUD_M1_201910.xlsx  has successfully read and concat.
Current Progress----   552  /  1212   ------ latest_pair_name:  GBPAUD | current_pair_name:  GBPAUD
Block 2 Execute
File  DAT_XLSX_GBPAUD_M1_201911.xlsx  has successfully read and concat.
Current Progress----   553  /  1212   ------ latest_pair_name:  GBPAUD | current_pair_name:  GBPCAD
Block 3 Execute
Opened database successfully
Table created successfully
GBPAUD has import to SQL Database.
File  DAT_XLSX_GBPCAD_M1_2007.xlsx  has successfully read.
Current Progress----   554  /  1212   ------ latest_pair_name:  GBPCAD | current_pair_name:  GBPCA

File  DAT_XLSX_GBPCHF_M1_201901.xlsx  has successfully read and concat.
Current Progress----   594  /  1212   ------ latest_pair_name:  GBPCHF | current_pair_name:  GBPCHF
Block 2 Execute
File  DAT_XLSX_GBPCHF_M1_201902.xlsx  has successfully read and concat.
Current Progress----   595  /  1212   ------ latest_pair_name:  GBPCHF | current_pair_name:  GBPCHF
Block 2 Execute
File  DAT_XLSX_GBPCHF_M1_201903.xlsx  has successfully read and concat.
Current Progress----   596  /  1212   ------ latest_pair_name:  GBPCHF | current_pair_name:  GBPCHF
Block 2 Execute
File  DAT_XLSX_GBPCHF_M1_201904.xlsx  has successfully read and concat.
Current Progress----   597  /  1212   ------ latest_pair_name:  GBPCHF | current_pair_name:  GBPCHF
Block 2 Execute
File  DAT_XLSX_GBPCHF_M1_201905.xlsx  has successfully read and concat.
Current Progress----   598  /  1212   ------ latest_pair_name:  GBPCHF | current_pair_name:  GBPCHF
Block 2 Execute
File  DAT_XLSX_GBPCHF_M1_201906.xlsx  has successfully read 

In [ ]:
#CADCHF has incomplete data. Remove from original file.